# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
api_key = '8fd5f0595355413cbe8dbe7085f10d03'

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,uthal,25.8072,66.6219,29.15,81,22,2.98,PK,1718929442
1,1,ribeira grande,38.5167,-28.7000,19.32,84,100,1.34,PT,1718929444
2,2,port-aux-francais,-49.3500,70.2167,2.32,69,41,8.43,TF,1718929445
3,3,puerto natales,-51.7236,-72.4875,-11.75,92,53,0.00,CL,1718929446
4,4,turtas,58.9417,69.1325,13.81,92,98,2.50,RU,1718929447


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    size='Humidity',
    hover_cols=['City', 'Country'],
    title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    cmap='Blues',  
    clim=(0, city_data_df['Humidity'].max()), 
    colorbar=True,
    frame_width=800,
    frame_height=500, 
    alpha=0.7,
    line_color='black',
    line_width=1.0,
    marker='circle',
    size_max=30, 
    fontsize={'title': 16, 'labels': 14, 'ticks': 12}, 
    toolbar='above', 
    padding=0.1,
)

# Display the map plot
map_plot

:Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Display sample data
ideal_cities_df.head()

C:\Users\railc\AppData\Local\Temp\ipykernel_32068\260414243.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,bethel,41.3712,-73.4140,26.76,71,0,4.12,US,1718929492
157,157,porto empedocle,37.2934,13.5264,26.51,42,0,1.70,IT,1718929642
160,160,broome,42.2506,-75.8330,25.43,88,0,2.06,US,1718929436
162,162,westport,41.1415,-73.3579,25.76,72,0,2.57,US,1718929648
178,178,live oak,36.9836,-121.9805,21.12,65,0,3.60,US,1718929671


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
39,bethel,US,41.3712,-73.4140,71,
157,porto empedocle,IT,37.2934,13.5264,42,
160,broome,US,42.2506,-75.8330,88,
162,westport,US,41.1415,-73.3579,72,
178,live oak,US,36.9836,-121.9805,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000  # Radius in meters
params = {
    "type": "hotel",
    "limit": 1,  # Limit to only one result (the nearest hotel)
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = {
        "around": {"lat": lat, "lon": lng},
        "radius": radius
    }
    params["bias"] = {"lat": lat, "lon": lng}

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
bethel - nearest hotel: No hotel found
porto empedocle - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
live oak - nearest hotel: No hotel found
venosa - nearest hotel: No hotel found
beruniy - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
cannonvale - nearest hotel: No hotel found
famagusta - nearest hotel: No hotel found
astrakhan - nearest hotel: No hotel found
arish - nearest hotel: No hotel found
alexandria - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
39,bethel,US,41.3712,-73.4140,71,No hotel found
157,porto empedocle,IT,37.2934,13.5264,42,No hotel found
160,broome,US,42.2506,-75.8330,88,No hotel found
162,westport,US,41.1415,-73.3579,72,No hotel found
178,live oak,US,36.9836,-121.9805,65,No hotel found
219,venosa,IT,40.9613,15.8149,37,No hotel found
315,beruniy,UZ,41.6911,60.7525,73,No hotel found
382,arraial do cabo,BR,-22.9661,-42.0278,87,No hotel found
407,cannonvale,AU,-20.2833,148.7000,72,No hotel found
443,famagusta,CY,35.1250,33.9500,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    size='Humidity',
    hover_cols=['City', 'Country', 'Hotel Name'], 
    title='City Humidity with Hotel Names',
    xlabel='Longitude',
    ylabel='Latitude',
    cmap='Blues', 
    colorbar=True,
    frame_width=800,
    frame_height=500,
    alpha=0.7,
    line_color='black',
    line_width=1.0,
    marker='circle',
    size_max=30,
    fontsize={'title': 16, 'labels': 14, 'ticks': 12},
    toolbar='above',
)

# Display the map plot
map_plot


:Scatter   [Lng]   (Lat,Humidity,City,Country)